# Lab 4: Enhancing RAG with Voyage-AI Reranking

This optional lab demonstrates how to integrate Voyage-AI's reranking capabilities into your RAG pipeline. Reranking helps to improve the precision of retrieved documents, ensuring that the most relevant information is passed to the LLM, leading to higher quality responses.

## Objectives
- Understand why reranking is beneficial in RAG.
- Implement Voyage-AI's reranking API to re-order search results.
- Observe the impact of reranking on the context provided to the LLM.

## Prerequisites
- Complete Lab 1, Lab 2, and Lab 3.
- Python environment set up with `pymongo`, `voyageai`, and `python-dotenv` installed.
- `.env` file containing `MONGO_URI` and `VOYAGE_API_KEY`.

## Step 1: Load Environment Variables and Initialize Clients

In [ ]:
from dotenv import load_dotenv
import os
import voyageai
from pymongo import MongoClient

# Load environment variables
load_dotenv()

# Initialize Voyage-AI Client
voyage_api_key = os.environ.get("VOYAGE_API_KEY")
if not voyage_api_key:
    raise ValueError("VOYAGE_API_KEY not found in .env file or environment variables.")
vo = voyageai.Client(api_key=voyage_api_key)

# Initialize MongoDB Client
mongo_uri = os.environ.get("MONGO_URI")
if not mongo_uri:
    raise ValueError("MONGO_URI not found in .env file or environment variables.")
client = MongoClient(mongo_uri)

# Select your database and collection
db = client['rag_db']
collection = db['documents']

print("Clients initialized successfully.")

## Step 2: Define User Query and Perform Initial Vector Search

We start with a user query and perform an initial vector search, potentially retrieving more documents than strictly needed, as reranking will help us select the best ones.

In [ ]:
user_query = "What are the latest security features?"
print(f"\nUser Query: {user_query}")

print("Generating query embedding with Voyage-AI...")
try:
    query_embedding_response = vo.embed(
        texts=[user_query],
        model="voyage-large-2", 
        input_type="query" 
    )
    query_embedding = query_embedding_response.embeddings[0]
    print("Query embedding generated.")
except Exception as e:
    print(f"Error generating query embedding: {e}")
    exit()

pipeline = [
  {
    '$vectorSearch': {
      'queryVector': query_embedding,
      'path': 'embedding',          
      'numCandidates': 100,         # Search more candidates for reranking
      'limit': 10,                  # Retrieve top 10 for reranking
      'index': 'default'            
    }
  },
  {
    '$project': {
      'text_chunk': 1,
      'source': 1,
      'score': { '$meta': 'vectorSearchScore' },
      '_id': 0 
    }
  }
]

print("Performing initial vector search in MongoDB Atlas...")
initial_retrieved_documents = list(collection.aggregate(pipeline))

if initial_retrieved_documents:
    print(f"Retrieved {len(initial_retrieved_documents)} initial documents for reranking.")
    for i, doc in enumerate(initial_retrieved_documents):
        print(f"  {i+1}. Score: {doc['score']:.4f}, Source: {doc['source']}, Text: {doc['text_chunk'][:50]}...")
else:
    print("No initial documents found.")
    exit()

## Step 3: Rerank Documents with Voyage-AI

We pass the original `user_query` and the `text_chunk`s from our initial retrieval to Voyage-AI's reranker. It will return a new set of scores indicating how relevant each document is to the query.

In [ ]:
documents_to_rerank = [doc['text_chunk'] for doc in initial_retrieved_documents]

print("\nReranking documents with Voyage-AI...")
try:
    rerank_result = vo.rerank(
        query=user_query,
        documents=documents_to_rerank,
        model="rerank-lite-1" # Or another reranking model
    )

    # Sort the original documents based on the new relevance scores
    reranked_documents_with_scores = sorted(
        zip(initial_retrieved_documents, rerank_result.results),
        key=lambda x: x[1].relevance_score, 
        reverse=True
    )

    print(f"Reranked {len(reranked_documents_with_scores)} documents.")
    print("Top reranked documents:")
    for i, (original_doc, reranked_item) in enumerate(reranked_documents_with_scores[:5]): # Show top 5
        print(f"  {i+1}. Rerank Score: {reranked_item.relevance_score:.4f}, Original Score: {original_doc['score']:.4f}, Source: {original_doc['source']}, Text: {original_doc['text_chunk'][:50]}...")

except Exception as e:
    print(f"Error reranking documents: {e}")
    exit()

## Step 4: Build Augmented Prompt with Reranked Context

Now we take the top documents after reranking and use them to build the context for our LLM prompt.

In [ ]:
top_n_after_rerank = 3 # Choose how many top reranked documents to send to the LLM
final_context_chunks = [doc[0]['text_chunk'] for doc in reranked_documents_with_scores[:top_n_after_rerank]]
context_reranked = "\n".join(final_context_chunks)

print("\n--- Reranked Context for LLM ---")
print(context_reranked)
print("-------------------------------")

if not context_reranked:
    print("Warning: No reranked context was available.")

In [ ]:
if context_reranked:
    llm_prompt_reranked = f"""
You are a helpful assistant. Answer the user's question based on the provided context only.
If you cannot find the answer in the context, politely state that the information is not available.

Context:
{context_reranked}

Question: {user_query}

Answer:
"""
else:
    llm_prompt_reranked = f"""
You are a helpful assistant. I couldn't find relevant information for the following question.
Please state that the information is not available in the provided knowledge base.

Question: {user_query}

Answer:
"""

print("\n--- LLM Augmented Prompt (with Reranking) ---")
print(llm_prompt_reranked)
print("------------------------------------------")

# Compare this prompt's context with the one from Lab 3. The ordering and relevance
# of the context chunks might be improved due to reranking.

## Conclusion

Reranking with Voyage-AI provides an effective way to refine the search results before feeding them to an LLM, potentially leading to more accurate and relevant responses in your RAG application. This is a crucial step for optimizing performance in real-world scenarios.

In [ ]:
# Don't forget to close the MongoDB client connection
client.close()
print("MongoDB client connection closed.")